# 0.0 Hämta data
- Fejkdata från microsoft. Taget fårn https://blog.fabric.microsoft.com/sv-SE/blog/prebuilt-azure-ai-services-in-fabric-2/

In [1]:
# Ladda data till pandas dataframe
df = spark.read.format("csv").\
     option("header","true").\
     load("Files/hotel_reviews/raw/hotel_reviews_demo.csv").\
     toPandas()
display(df)

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2faac4c3-8270-4d98-be0e-fa12492811bf)

# 1.0 Translate
- Installera translator SDK

In [2]:
pip install azure-ai-translation-text==1.0.0b1

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 4, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 2.0.0 requires sentencepiece, which is not installed.
sentence-transformers 2.0.0 requires torchvision, which is not installed.
dash 2.14.0 requires Flask<2.3.0,>=1.0.4, but you have flask 3.0.0 which is incompatible.
dash 2.14.0 requires Werkzeug<2.3.0, but you have werkzeug 3.0.1 which is incompatible.
tensorflow 2.12.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.
Note: you may need t

In [3]:
# Importera nödvändiga bibliotek 
from azure.ai.translation.text import TextTranslationClient, TranslatorCredential
from azure.ai.translation.text.models import InputTextItem
from azure.core.exceptions import HttpResponseError

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 5, Finished, Available, Finished)

In [4]:
# Hämta credentials
from notebookutils.mssparkutils.credentials import getSecret

KEYVAULT_ENDPOINT ="https://lab-key-valut-axel.vault.azure.net/"
#Azure AI Services Key
AI_SERVICES_KEY = getSecret(KEYVAULT_ENDPOINT,"key-ai-services-axel-lab")

REGION = "swedencentral"
ENDPOINT = "https://api.cognitive.microsofttranslator.com/"

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 6, Finished, Available, Finished)

In [5]:
# Starta Translate clienten
credential = TranslatorCredential(AI_SERVICES_KEY, REGION)
text_translator = TextTranslationClient(endpoint=ENDPOINT, credential=credential)

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 7, Finished, Available, Finished)

## Uppfigt 1.0
- I kolumnen "reviews_text" finns hotellrecensioner på flera olika språk. Använd ett translate clitenten för att identifiera språket i recensionerna och översätt dem till svenska i en ny kolumn, "review_svenska"

### Exemepl

In [6]:
try:
    to_language = ["cs", "es", "de"]
    input_text_elements = [ InputTextItem(text = "This is a test") ]

    response = text_translator.translate(content=input_text_elements, to=to_language)
    translation = response[0] if response else None

    if translation:
        detected_language = translation.detected_language
        if detected_language:
            print(
                f"Detected languages of the input text: {detected_language.language} with score: {detected_language.score}."
            )
        for translated_text in translation.translations:
            print(f"Text was translated to: '{translated_text.to}' and the result is: '{translated_text.text}'.")

except HttpResponseError as exception:
    if exception.error is not None:
        print(f"Error Code: {exception.error.code}")
        print(f"Message: {exception.error.message}")

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 8, Finished, Available, Finished)

Detected languages of the input text: en with score: 1.0.
Text was translated to: 'cs' and the result is: 'Tohle je test'.
Text was translated to: 'es' and the result is: 'Esto es una prueba'.
Text was translated to: 'de' and the result is: 'Dies ist ein Test'.


In [11]:
input_text_elements = [InputTextItem(text=df['reviews_text'].iloc[0])]
text_translator.translate(content=input_text_elements,to=['sv'])

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 13, Finished, Available, Finished)

[{'detectedLanguage': {'language': 'en', 'score': 1.0}, 'translations': [{'text': 'Jag blev besviken på det här hotellet - det når definitivt inte upp till den normala Sheraton-standarden. Internetavgifterna var alltför dyra och systemet de använde var svårt att använda. Överlag var hotellrummen daterade och luftkonditioneringen fungerade inte.', 'to': 'sv'}]}]

In [12]:
# List to store translated reviews
translated_reviews = []

# Loop through each review
for review in df['reviews_text']:
    # Translate the review and append the translated text to the list
    input_text_element = [InputTextItem(text=review)]
    translated_text = text_translator.translate(content=input_text_element, to=['sv'])
    translated_reviews.append(translated_text[0].translations[0].text)  # Assuming the response contains translations

# Add the translated reviews as a new column in the DataFrame
df['translated_reviews'] = translated_reviews

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 14, Finished, Available, Finished)

In [14]:
display(df)

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, dc5b53a0-c871-4811-8e78-053382e8947f)

# 2.0 AI Language 
- Installera textanalytics SDK

In [15]:
pip install azure-ai-textanalytics==5.2.0

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 17, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 3.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [16]:
# Hämta credentials
from notebookutils.mssparkutils.credentials import getSecret

KEYVAULT_ENDPOINT ="https://lab-key-valut-axel.vault.azure.net/"
#Azure AI Services Key
AI_SERVICES_KEY = getSecret(KEYVAULT_ENDPOINT,"key-ai-services-axel-lab")

ENDPOINT = "https://axel-ai-services-lab.cognitiveservices.azure.com/"

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 18, Finished, Available, Finished)

In [17]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 19, Finished, Available, Finished)

In [18]:
credential = AzureKeyCredential(AI_SERVICES_KEY)
text_analytics_client = TextAnalyticsClient(endpoint=ENDPOINT,credential=credential)

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 20, Finished, Available, Finished)

## Uppfigt 2.0
- Identifiera nyckel fraser från varje review och spara i en ny kolumn reviews_keyphrases

### Exempel

In [19]:
try:
    documents = ["Dr. Smith has a very modern medical office, and she has great staff."]

    response = text_analytics_client.extract_key_phrases(documents = documents)[0]

    if not response.is_error:
        print("\tKey Phrases:")
        for phrase in response.key_phrases:
            print("\t\t", phrase)
    else:
        print(response.id, response.error)

except Exception as err:
    print("Encountered exception. {}".format(err))

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 21, Finished, Available, Finished)

	Key Phrases:
		 modern medical office
		 Dr. Smith
		 great staff


In [29]:
# List to store translated reviews
key_phrases_reviews = []

# Loop through each review
for review in df['reviews_text']:

    documents = [review]
    response = text_analytics_client.extract_key_phrases(documents = documents)[0]
    melted_string = ' '.join(response.key_phrases)
    key_phrases_reviews.append(melted_string)  # Assuming the response contains translations

# Add the translated reviews as a new column in the DataFrame
df['key_phrases_reviews'] = key_phrases_reviews

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 31, Finished, Available, Finished)

In [30]:
display(df)

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 32, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, bd2549c3-4715-4bb5-a685-cec1767ae0ac)

## Uppgift 2.1
- Extrahera alla städer som nämns i varje review. Spara städerna i en ny kolumn. Om ingen stad nämns lämna fältet tomt.

### Exempel 

In [31]:
try:
    documents = ["I had a wonderful trip to Seattle last week."]
    result = text_analytics_client.recognize_entities(documents = documents)[0]

    print("Named Entities:\n")
    for entity in result.entities:
        print("\tText: \t", entity.text, "\tCategory: \t", entity.category, "\tSubCategory: \t", entity.subcategory,
                    "\n\tConfidence Score: \t", round(entity.confidence_score, 2), "\tLength: \t", entity.length, "\tOffset: \t", entity.offset, "\n")

except Exception as err:
    print("Encountered exception. {}".format(err))

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 33, Finished, Available, Finished)

Named Entities:

	Text: 	 trip 	Category: 	 Event 	SubCategory: 	 None 
	Confidence Score: 	 0.82 	Length: 	 4 	Offset: 	 18 

	Text: 	 Seattle 	Category: 	 Location 	SubCategory: 	 City 
	Confidence Score: 	 1.0 	Length: 	 7 	Offset: 	 26 

	Text: 	 last week 	Category: 	 DateTime 	SubCategory: 	 DateRange 
	Confidence Score: 	 1.0 	Length: 	 9 	Offset: 	 34 



- Hjälp med dokumentation
- https://learn.microsoft.com/en-us/azure/ai-services/language-service/named-entity-recognition/concepts/named-entity-categories?tabs=ga-api

In [40]:
# List to store reviews with city mentions
city_reviews = []

# Loop through each review in the DataFrame
for review in df['reviews_text']:
    documents = [review]
    
    # Use entity recognition to extract cities
    response = text_analytics_client.recognize_entities(documents=documents)[0]
    
    city_list = []
    
    # Loop through entities and check for subcategory 'City'
    for entity in response.entities:
        if entity.subcategory == 'City':
            city_list.append(entity.text)  # Assuming 'entity.text' gives the entity name

    city_reviews.append(city_list)

# Add the extracted cities as a new column in the DataFrame
df['city_mentions'] = city_reviews

# Print the city_mentions list to verify the results
print(city_reviews)

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 42, Finished, Available, Finished)

[[], [], [], [], [], [], [], ['Taormina'], [], [], [], [], [], [], [], [], ['La Quinta'], [], [], [], [], [], [], ['Trocadero'], [], [], [], [], [], [], [], ['Houston'], [], [], [], [], [], ['hampton'], [], [], [], [], ['north Tahoe'], [], [], [], [], ['San Antonio'], ['Grand Rapids'], [], [], [], [], ['Nice'], [], [], ['Fort Wayne'], [], ['Newburghdine'], [], [], ['Beach Cove'], [], [], [], ['Telluride'], [], [], [], ['Yellowstone'], [], ['Jackson'], [], [], [], [], [], [], ['Offenbach'], ['Annapolis'], ['Houston'], [], ['Santa Cruz'], [], [], [], [], [], ['Rockford'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['San Francisco'], [], [], [], [], [], [], [], ['Hopkinsville'], [], [], [], [], [], [], [], ['Venice', 'Venice', 'Venice', 'Venice'], [], [], [], [], ['Grove City'], [], [], ['Marriott', 'Miami'], [], ['Phoenix'], [], [], [], [], [], ['Weston', 'Miami'], ['Rock'], [], [], [], [], [], ['Tahoe'], [], [], ['Sicily', 'Rome', 'Palermo', 'Taormina', 'Siracuse', 

## Uppgift 2.2a
- Använd Personally Identifiable Information (PII) för att se om någon har lämnat personuppfigter i sin review.

### Exempel
- Mer dokumentation: https://learn.microsoft.com/en-us/python/api/azure-ai-textanalytics/azure.ai.textanalytics.piientitycategory?view=azure-python 

In [41]:
documents = [
        "The employee's SSN is 859-98-0987.",
        "The employee's phone number is 555-555-5555."
    ]
response = text_analytics_client.recognize_pii_entities(documents=documents, language="en")
result = [doc for doc in response if not doc.is_error]
for doc in result:
    print("Redacted Text: {}".format(doc.redacted_text))
    for entity in doc.entities:
        print("Entity: {}".format(entity.text))
        print("\tCategory: {}".format(entity.category))
        print("\tConfidence Score: {}".format(entity.confidence_score))
        print("\tOffset: {}".format(entity.offset))
        print("\tLength: {}".format(entity.length))

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 43, Finished, Available, Finished)

Redacted Text: The ********'s SSN is ***********.
Entity: employee
	Category: PersonType
	Confidence Score: 0.99
	Offset: 4
	Length: 8
Entity: 859-98-0987
	Category: USSocialSecurityNumber
	Confidence Score: 0.85
	Offset: 22
	Length: 11
Redacted Text: The ********'s phone number is ************.
Entity: employee
	Category: PersonType
	Confidence Score: 0.99
	Offset: 4
	Length: 8
Entity: 555-555-5555
	Category: PhoneNumber
	Confidence Score: 0.8
	Offset: 31
	Length: 12


## Uppgift 2.2b
- Planetera några personuppgifter och se om du kan hitta dem.
- Tex, Email, personnummer eller kanske ett svensk passnummer? 

## Uppgift 2.3
- Identifiera positiva och negativa reviews
- Hjälp: https://learn.microsoft.com/en-us/azure/ai-services/language-service/sentiment-opinion-mining/language-support

### Exempel
![image-alt-text](https://learn.microsoft.com/en-us/azure/ai-services/language-service/sentiment-opinion-mining/media/opinion-mining.png)

In [20]:
documents = [
    "The room was great, but the staff was unfriendly"]

result = text_analytics_client.analyze_sentiment(documents, show_opinion_mining=True)
doc_result = [doc for doc in result if not doc.is_error]

positive_reviews = [doc for doc in doc_result if doc.sentiment == "positive"]
negative_reviews = [doc for doc in doc_result if doc.sentiment == "negative"]

positive_mined_opinions = []
mixed_mined_opinions = []
negative_mined_opinions = []

for document in doc_result:
    print("Document Sentiment: {}".format(document.sentiment))
    print("Overall scores: positive={0:.2f}; neutral={1:.2f}; negative={2:.2f} \n".format(
       document.confidence_scores.positive,
       document.confidence_scores.neutral,
       document.confidence_scores.negative,
    ))
    for sentence in document.sentences:
        print("Sentence: {}".format(sentence.text))
        print("Sentence sentiment: {}".format(sentence.sentiment))
        print("Sentence score:\nPositive={0:.2f}\nNeutral={1:.2f}\nNegative={2:.2f}\n".format(
           sentence.confidence_scores.positive,
           sentence.confidence_scores.neutral,
           sentence.confidence_scores.negative,
        ))
        for mined_opinion in sentence.mined_opinions:
            target = mined_opinion.target
            print("......'{}' target '{}'".format(target.sentiment, target.text))
            print("......Target score:\n......Positive={0:.2f}\n......Negative={1:.2f}\n".format(
               target.confidence_scores.positive,
               target.confidence_scores.negative,
            ))
            for assessment in mined_opinion.assessments:
                print("......'{}' assessment '{}'".format(assessment.sentiment, assessment.text))
                print("......Assessment score:\n......Positive={0:.2f}\n......Negative={1:.2f}\n".format(
                   assessment.confidence_scores.positive,
                   assessment.confidence_scores.negative,
                ))
        print("\n")
    print("\n")

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 22, Finished, Available, Finished)

Document Sentiment: negative
Overall scores: positive=0.00; neutral=0.00; negative=0.99 

Sentence: The room was great, but the staff was unfriendly
Sentence sentiment: negative
Sentence score:
Positive=0.00
Neutral=0.00
Negative=0.99

......'positive' target 'room'
......Target score:
......Positive=1.00
......Negative=0.00

......'positive' assessment 'great'
......Assessment score:
......Positive=1.00
......Negative=0.00

......'negative' target 'staff'
......Target score:
......Positive=0.00
......Negative=1.00

......'negative' assessment 'unfriendly'
......Assessment score:
......Positive=0.00
......Negative=1.00







In [46]:
documents = [
    "The room was great, but the staff was unfriendly"]

result = text_analytics_client.analyze_sentiment(documents, show_opinion_mining=True)
print(result[0].sentiment)

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 48, Finished, Available, Finished)

negative


In [ ]:
# List to store reviews with city mentions
sentiment_reviews = []

# Loop through each review in the DataFrame
for review in df['reviews_text']:
    documents = [review]
    
    # Use entity recognition to extract cities
    response = text_analytics_client.analyze_sentiment(documents=documents)[0]
    sentiment_reviews.append(result[0].sentiment)

# Add the extracted cities as a new column in the DataFrame
df['city_mentions'] = city_reviews

# Print the city_mentions list to verify the results
print(city_reviews)

# 3.0 Document Intelligence 
- Installera documentintelligence SDK

In [33]:
pip install azure-ai-documentintelligence==1.0.0b4

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 35, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.5/99.5 kB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.4/197.4 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: azure-core
    Found existing installation: azure-core 1.29.4
    Uninstalling azure-core-1.29.4:
      Successfully uninstalled azure-core-1.29.4
Note: you may need to restart the kernel to use updated packages.


In [34]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult,AnalyzeDocumentRequest

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 36, Finished, Available, Finished)

In [35]:
# Hämta credentials
from notebookutils.mssparkutils.credentials import getSecret

KEYVAULT_ENDPOINT ="https://lab-key-valut-axel.vault.azure.net/"
#Azure AI Services Key
KEY = getSecret(KEYVAULT_ENDPOINT,"key-document-intelligence-axel-lab")

ENDPOINT = "https://axel-document-intelligence-lab.cognitiveservices.azure.com/"

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 37, Finished, Available, Finished)

## Uppgift 3.0
- Vi behöver mer reviews. Dock finns dessa endast i png och pdf format.
- Extrahetra texten från följande bild med hjälp av docuument intelligence 
![image-alt-text](https://travelerwp.com/wp-content/uploads/2023/02/How-to-ask-for-Tripadvisor-reviews-Image-10.png)
- url: https://travelerwp.com/wp-content/uploads/2023/02/How-to-ask-for-Tripadvisor-reviews-Image-10.png

### Exempel

In [54]:
# helper functions
def get_words(page, line):
    result = []
    for word in page.words:
        if _in_span(word, line.spans):
            result.append(word)
    return result


def _in_span(word, spans):
    for span in spans:
        if word.span.offset >= span.offset and (word.span.offset + word.span.length) <= (span.offset + span.length):
            return True
    return False


def analyze_read():
    # sample document
    formUrl = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/rest-api/read.png"

    client = DocumentIntelligenceClient(
        endpoint=ENDPOINT, credential=AzureKeyCredential(KEY)
    )

    poller = client.begin_analyze_document(
        "prebuilt-read", AnalyzeDocumentRequest(url_source=formUrl
    ))
    result: AnalyzeResult = poller.result()

    print("----Languages detected in the document----")
    if result.languages is not None:
        for language in result.languages:
            print(f"Language code: '{language.locale}' with confidence {language.confidence}")

    print("----Styles detected in the document----")
    if result.styles:
        for style in result.styles:
            if style.is_handwritten:
                print("Found the following handwritten content: ")
                print(",".join([result.content[span.offset : span.offset + span.length] for span in style.spans]))
            if style.font_style:
                print(f"The document contains '{style.font_style}' font style, applied to the following text: ")
                print(",".join([result.content[span.offset : span.offset + span.length] for span in style.spans]))

    for page in result.pages:
        print(f"----Analyzing document from page #{page.page_number}----")
        print(f"Page has width: {page.width} and height: {page.height}, measured with unit: {page.unit}")

        #if page.lines:
        #    for line_idx, line in enumerate(page.lines):
        #        words = get_words(page, line)
        #        print(
        #            f"...Line # {line_idx} has {len(words)} words and text '{line.content}' within bounding polygon '{line.polygon}'"
        #        )

                #for word in words:
                #    print(f"......Word '{word.content}' has a confidence of {word.confidence}")

        #if page.selection_marks:
        #    for selection_mark in page.selection_marks:
        #        print(
        #            f"...Selection mark is '{selection_mark.state}' within bounding polygon "
        #            f"'{selection_mark.polygon}' and has a confidence of {selection_mark.confidence}"
        #        )

    if result.paragraphs:
        print(f"----Detected #{len(result.paragraphs)} paragraphs in the document----")
        for paragraph in result.paragraphs:
            if paragraph.role == None:
                print(f"Found paragraph with role: '{paragraph.role}' within {paragraph.bounding_regions} bounding region")
                print(f"...with content: '{paragraph.content}'")

        result.paragraphs.sort(key=lambda p: (p.spans.sort(key=lambda s: s.offset), p.spans[0].offset))
        print("-----Print sorted paragraphs-----")
        for idx, paragraph in enumerate(result.paragraphs):
            print(
                f"...paragraph:{idx} with offset: {paragraph.spans[0].offset} and length: {paragraph.spans[0].length}"
            )

    print("----------------------------------------")

analyze_read()

StatementMeta(, fa526353-5001-4204-b384-c1bc9673a3e2, 56, Finished, Available, Finished)

----Languages detected in the document----
----Styles detected in the document----
----Analyzing document from page #1----
Page has width: 915 and height: 1190, measured with unit: pixel
----Detected #16 paragraphs in the document----
Found paragraph with role: 'None' within [{'pageNumber': 1, 'polygon': [259, 55, 826, 55, 826, 134, 259, 134]}] bounding region
...with content: 'While healthcare is still in the early stages of its Al journey, we are seeing pharmaceutical and other life sciences organizations making major investments in Al and related technologies."'
Found paragraph with role: 'None' within [{'pageNumber': 1, 'polygon': [258, 151, 637, 151, 637, 167, 258, 167]}] bounding region
...with content: 'TOM LAWRY | National Director for Al, Health and Life Sciences | Microsoft'
Found paragraph with role: 'None' within [{'pageNumber': 1, 'polygon': [75, 240, 441, 240, 441, 403, 75, 403]}] bounding region
...with content: 'As pharmaceutical and other life sciences organizations in

## Uppgift 3.1
- Det finns även en pdf hämtat från webben med hotel reviews av grand hotel från trip advisor. Dessa ska också läsas in.
- Uppgiften är att med hjälp av Document Intelligence extrahera text från varje omdömde i pdf:en
- Hjälp med dokumentation: https://pypi.org/project/azure-ai-documentintelligence/#analyze-documents-result-in-pdf  

**Kod för att läsa in pdf:en och skicka den till document intelligence**

In [ ]:
# Path to your PDF file
pdf_file_path = '/lakehouse/default/Files/grandhotel_reviews.pdf'

# Initialize Document Analysis Client with endpoint and key
client = DocumentIntelligenceClient(
    endpoint=ENDPOINT, credential=AzureKeyCredential(KEY)
    )

# Open the PDF file for reading in binary mode
with open(pdf_file_path, "rb") as f:
    poller = client.begin_analyze_document(
    "prebuilt-layout", analyze_request=f, content_type="application/octet-stream"
    )
result: AnalyzeResult = poller.result()